# ****Laboratorio 7****
### Integrantes:
- Pedro Pablo Arriola Jimenez (20188)
- Marco Pablo Orozco Saravia (20857)
- Santiago Taracena Puga (20017)

### Instrucciones:
- Deben unirse a uno de los grupos de Canvas de nombre “Laboratorio 7 #”, donde N es un número entre 1 y 23. Los grupos pueden ser de 2 o 3 personas.
- Esta actividad debe realizarse en grupos.
- Sólo es necesario que una persona del grupo suba el trabajo a Canvas.
- No se permitirá ni se aceptará cualquier indicio de copia. De presentarse, se procederá según el reglamento correspondiente.

### Task 1 - Práctica

Considere las arquitecturas conversadas durante la clase, con ello realice una implementación de dos arquitecturas usando PyTorch

1. Implemente la arquitectura de LeNet-5 para resolver el problema de clasificación del daset de dígitos escritos a mano llamado mnist dataset
2. Implemente la arquitectura de AlexNet para resolver el problema de clasificación usando el dataset de imagenes llamado CIFAR10 dataset.

Para cada implementación defina y justifique (dentro del notebook) una métrica de desempeño. Además responda (en su notebook), recuerde justificar y/o expandir su respuesta:

a. ¿Cuál es la diferencia principal entre ambas arquitecturas?

b. ¿Podría usarse LeNet-5 para un problema como el que resolvió usando AlexNet? ¿Y viceversa?

c. Indique claramente qué le pareció más interesante de cada arquitectura

### Parte 1 - LeNet-5

In [1]:
# Importar las bibliotecas necesarias
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

In [2]:
# Cargar y preprocesar el conjunto de datos MNIST
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 21206265.85it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 55313102.20it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 3782652.36it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 4469856.59it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
# Verificar si CUDA está disponible y configurar el dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
# Se define la arquitectura LeNet-5
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        # Definir la primera capa convolucional
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)
        self.act1 = nn.Tanh()
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)
        
        # Definir la segunda capa convolucional
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)
        
        # Definir las capas completamente conectadas
        self.fc1 = nn.Linear(16*5*5, 120)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(120, 84)
        self.act4 = nn.Tanh()
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Definir la propagación hacia adelante
        x = self.pool1(self.act1(self.conv1(x)))
        x = self.pool2(self.act2(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = self.act3(self.fc1(x))
        x = self.act4(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
# Instanciar el modelo y moverlo a la GPU si está disponible
model = LeNet5().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
# Entrenar el modelo
for epoch in range(10):
    print(f"Comenzando Época {epoch+1}...")
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if (batch_idx + 1) % 100 == 0:
            print(f"Época [{epoch+1}/{10}], Paso [{batch_idx+1}/{len(train_loader)}], Pérdida: {loss.item():.4f}")
    
    print(f"Finalizando Época {epoch+1}...")

# Guardar el modelo entrenado
torch.save(model.state_dict(), "lenet5_mnist.pth")

Comenzando Época 1...
Época [1/10], Paso [100/938], Pérdida: 0.1782
Época [1/10], Paso [200/938], Pérdida: 0.0311
Época [1/10], Paso [300/938], Pérdida: 0.1647
Época [1/10], Paso [400/938], Pérdida: 0.0115
Época [1/10], Paso [500/938], Pérdida: 0.1288
Época [1/10], Paso [600/938], Pérdida: 0.0251
Época [1/10], Paso [700/938], Pérdida: 0.0632
Época [1/10], Paso [800/938], Pérdida: 0.0754
Época [1/10], Paso [900/938], Pérdida: 0.1118
Finalizando Época 1...
Comenzando Época 2...
Época [2/10], Paso [100/938], Pérdida: 0.0168
Época [2/10], Paso [200/938], Pérdida: 0.0056
Época [2/10], Paso [300/938], Pérdida: 0.0164
Época [2/10], Paso [400/938], Pérdida: 0.0210
Época [2/10], Paso [500/938], Pérdida: 0.0415
Época [2/10], Paso [600/938], Pérdida: 0.0292
Época [2/10], Paso [700/938], Pérdida: 0.1321
Época [2/10], Paso [800/938], Pérdida: 0.0453
Época [2/10], Paso [900/938], Pérdida: 0.0369
Finalizando Época 2...
Comenzando Época 3...
Época [3/10], Paso [100/938], Pérdida: 0.0136
Época [3/10], 

In [8]:
# Función para calcular las métricas
def calculate_metrics(target, pred):
    precision = precision_score(target, pred, average='macro')
    recall = recall_score(target, pred, average='macro')
    f1 = f1_score(target, pred, average='macro')
    confusion = confusion_matrix(target, pred)
    return precision, recall, f1, confusion

In [9]:
# Cargar el modelo entrenado
model = LeNet5().to(device)
model.load_state_dict(torch.load("lenet5_mnist.pth"))
model.eval()

# Inicializar listas para predicciones y etiquetas verdaderas
all_preds = []
all_targets = []

# Realizar la evaluación
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True).squeeze()
        all_preds.extend(pred.cpu().numpy())
        all_targets.extend(target.cpu().numpy())

# Calcular y mostrar las métricas
precision, recall, f1, confusion = calculate_metrics(all_targets, all_preds)
accuracy = sum(p == t for p, t in zip(all_preds, all_targets)) / len(all_targets)
print(f"Resumen de Evaluación:")
print(f"Accuracy: {accuracy:.2%}")
print(f"Precision: {precision:.2%}")
print(f"Recall: {recall:.2%}")
print(f"F1-Score: {f1:.2%}")
print(f"Confusion Matrix: \n{confusion}\n")

Resumen de Evaluación:
Accuracy: 98.55%
Precision: 98.56%
Recall: 98.53%
F1-Score: 98.54%
Confusion Matrix: 
[[ 973    0    0    0    0    0    1    2    2    2]
 [   1 1130    1    0    0    0    1    1    1    0]
 [   4    1 1016    0    1    0    0    7    2    1]
 [   0    0    1  990    0    4    0    8    1    6]
 [   0    0    0    0  969    0    0    1    0   12]
 [   4    0    0    3    1  872    4    2    3    3]
 [   7    4    1    1    2    0  939    0    3    1]
 [   0    4    4    0    0    0    0 1017    1    2]
 [   4    0    1    4    0    2    0    3  958    2]
 [   1    2    0    0    5    3    0    6    1  991]]



### Parte 2 - AlexNet

### Task 2 - Teoría
Responda claramente y con una extensión adecuada las siguientes preguntas:
1. Investigue e indique en qué casos son útiles las siguientes arquitecturas, agregue imagenes si esto le ayuda a una mejor comprensión

- GoogleNet (Inception)

- DenseNet (Densely Connected Convolutional Networks)

- MobileNet

-  EfficientNet

2. ¿Cómo la arquitectura de transformers puede ser usada para image recognition?